In [82]:
import pandas as pd
import matplotlib as plt

from sklearn_pandas import DataFrameMapper

# Packages for NLP
import nltk
from nltk.corpus import stopwords
import regex as re

# Packages for sentiment analysis
from textblob import TextBlob

# Packages for visualisation 
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer

# Packages for data 
import pandas as pd
import numpy as np
import pickle
from collections import Counter

# Packages for machine learning modelling
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report,confusion_matrix, ConfusionMatrixDisplay, accuracy_score
# precision_score, recall_score, f1_score

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

# Packages for sentiment analysis
from textblob import TextBlob

# Packages for visualisation 
import matplotlib.pyplot as plt

# Packages for MLP
import nltk


from sklearn.preprocessing import MinMaxScaler

# Read

In [83]:
train_data = pd.read_csv("../Data/train_data.csv")
val_data = pd.read_csv("../Data/validation_data.csv")
test_data = pd.read_csv("../Data/test_data.csv")

X_train_text = train_data["text_preprocessed"].values
y_train = train_data["class_label"].values

# X_val_text = val_data["text_preprocessed"].values
# y_val = val_data["class_label"].values

X_test_text = test_data["text_preprocessed"].values
y_test = test_data["class_label"].values

train_data_AF = pd.read_csv('train_data_with_added_features.csv')
test_data_AF = pd.read_csv("test_data_with_added_features.csv")

# Just TFIDF

In [84]:
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', ngram_range=(1,2), min_df=0.15)

X_train = tfidf_vectorizer.fit_transform(X_train_text)
# X_val = tfidf_vectorizer.transform(X_val_text)
X_test = tfidf_vectorizer.transform(X_test_text)

# Mapper

In [85]:
mapper = DataFrameMapper([
    (['char_count', 'word_count', 'sentence_count', 'num_unique_words', 'avg_sentence_length', 'num_punctuations', 'num_stopwords', 'num_words_in_quotes', 'num_nouns', 'num_verbs', 'num_adjectives', 'num_discourse_relations', 'textblob_sentiment'], None), 
    ('text_preprocessed',tfidf_vectorizer)
])

In [86]:
X_train_AF = mapper.fit_transform(train_data_AF)
print(X_train_AF.shape)

X_test_AF = mapper.transform(test_data_AF)
print(X_test_AF.shape)

/Users/joshuawong/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


(39060, 147)
(13021, 147)


# Model

## Default

In [87]:
naive_bayes_clf = BernoulliNB()
naive_bayes_clf.fit(X_train, y_train)

y_pred = naive_bayes_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85      6361
           1       0.86      0.85      0.86      6660

    accuracy                           0.85     13021
   macro avg       0.85      0.85      0.85     13021
weighted avg       0.85      0.85      0.85     13021



In [88]:
# Logreg
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_train, y_train)

y_test_pred = log_reg_clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90      6361
           1       0.90      0.92      0.91      6660

    accuracy                           0.91     13021
   macro avg       0.91      0.91      0.91     13021
weighted avg       0.91      0.91      0.91     13021



## Default Scaled

In [89]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

TypeError: MinMaxScaler does not support sparse input. Consider using MaxAbsScaler instead.

In [ ]:
naive_bayes_clf = BernoulliNB()
naive_bayes_clf.fit(X_train, y_train)

y_pred = naive_bayes_clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
# Logreg
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_train, y_train)

y_test_pred = log_reg_clf.predict(X_test)
print(classification_report(y_test, y_test_pred))

## Added Features

In [91]:
naive_bayes_clf = BernoulliNB()
naive_bayes_clf.fit(X_train_AF, y_train)

y_pred = naive_bayes_clf.predict(X_test_AF)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      6361
           1       0.86      0.85      0.85      6660

    accuracy                           0.85     13021
   macro avg       0.85      0.85      0.85     13021
weighted avg       0.85      0.85      0.85     13021



In [90]:
# Logreg
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_train_AF, y_train)

y_test_pred = log_reg_clf.predict(X_test_AF)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.71      0.63      0.67      6361
           1       0.68      0.76      0.72      6660

    accuracy                           0.69     13021
   macro avg       0.70      0.69      0.69     13021
weighted avg       0.70      0.69      0.69     13021



/Users/joshuawong/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Added Features scaled

In [92]:
X_train_AF = scaler.fit_transform(X_train_AF)
X_test_AF = scaler.fit_transform(X_test_AF)

In [93]:
naive_bayes_clf = BernoulliNB()
naive_bayes_clf.fit(X_train_AF, y_train)

y_pred = naive_bayes_clf.predict(X_test_AF)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.86      0.85      6361
           1       0.86      0.84      0.85      6660

    accuracy                           0.85     13021
   macro avg       0.85      0.85      0.85     13021
weighted avg       0.85      0.85      0.85     13021



In [94]:
# Logreg
log_reg_clf = LogisticRegression()
log_reg_clf.fit(X_train_AF, y_train)

y_test_pred = log_reg_clf.predict(X_test_AF)
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.92      0.89      0.91      6361
           1       0.90      0.93      0.91      6660

    accuracy                           0.91     13021
   macro avg       0.91      0.91      0.91     13021
weighted avg       0.91      0.91      0.91     13021



/Users/joshuawong/miniforge3/envs/tensorflow/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
